In [5]:
import pandas as pd
import random
import copy
import os
from tqdm import tqdm
import numpy as np
import re

In [6]:
all_data = pd.read_csv("overall_title_labeled.csv")
all_data['query'] = all_data.apply(lambda row:np.nan if pd.isna(row['query']) else re.sub(r' 2\. .*', '', row['query']), axis = 1)
all_data

,index,session_id,event_id,name,is_label,query
0,0,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,item_view,Satin Pajama sets shawl collar allover print n...,sample,NaN
1,1,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,item_view,Carter's 9-12 Months Girls Girls 0-24 mos dress,sample,NaN
2,2,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,item_view,Lonxu Pink With Pajama sets Black Trim Silk Sa...,sample,NaN
3,3,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,item_add_to_cart_tap,Pajama sets Lonxu Pink With Black Trim Silk Sa...,sample,NaN
4,4,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,buy_comp,Lonxu Pink With Black Trim Silk Satin Pajama s...,sample,NaN
...,...,...,...,...,...,...
214820,215058,fffc7a2257c114a4fbad5e1a4f5ec7ed1d840286b1d9ce...,item_view,Carter's Girls yellow/pink Girls 0-24 mos 24 M...,sample,NaN
214821,215059,fffc7a2257c114a4fbad5e1a4f5ec7ed1d840286b1d9ce...,item_view,bundle 3T Carter's Girls 2T-5T,sample,NaN
214822,215060,fffc7a2257c114a4fbad5e1a4f5ec7ed1d840286b1d9ce...,item_view,5T Carter's Girls 2T-5T Bundle of girls “play”...,sample,NaN
214823,215061,fffc7a2257c114a4fbad5e1a4f5ec7ed1d840286b1d9ce...,item_view,Toddler Girls 2T-5T girl Old Navy 3T dress bundle,sample,NaN


In [7]:
new_data = copy.deepcopy(all_data)
new_data['gen_data'] = new_data.apply(lambda row:[[row['event_id'],row['name']]] if row['is_label'] != "label" else [["label",row['query']]], axis = 1)
query_data = new_data.apply(lambda row:np.nan if (pd.isna(row['query']) or row['is_label'] == "label") else [["query",row['query']]], axis = 1)
query_filter = new_data['is_label'] == "query"
new_data['gen_data'][query_filter] = query_data[query_filter] + new_data['gen_data'][query_filter]
final_data = new_data[['session_id','gen_data']].explode("gen_data")
final_data

<ipython-input-7-4df0e83d73ab>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['gen_data'][query_filter] = query_data[query_filter] + new_data['gen_data'][query_filter]


,session_id,gen_data
0,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,"[item_view, Satin Pajama sets shawl collar all..."
1,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,"[item_view, Carter's 9-12 Months Girls Girls 0..."
2,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,"[item_view, Lonxu Pink With Pajama sets Black ..."
3,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,"[item_add_to_cart_tap, Pajama sets Lonxu Pink ..."
4,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,"[buy_comp, Lonxu Pink With Black Trim Silk Sat..."
...,...,...
214820,fffc7a2257c114a4fbad5e1a4f5ec7ed1d840286b1d9ce...,"[item_view, Carter's Girls yellow/pink Girls 0..."
214821,fffc7a2257c114a4fbad5e1a4f5ec7ed1d840286b1d9ce...,"[item_view, bundle 3T Carter's Girls 2T-5T]"
214822,fffc7a2257c114a4fbad5e1a4f5ec7ed1d840286b1d9ce...,"[item_view, 5T Carter's Girls 2T-5T Bundle of ..."
214823,fffc7a2257c114a4fbad5e1a4f5ec7ed1d840286b1d9ce...,"[item_view, Toddler Girls 2T-5T girl Old Navy ..."


In [8]:
final_data_copy = copy.deepcopy(final_data)
final_data_copy['action_type'] = final_data_copy['gen_data'].apply(lambda x:x[0])
final_data_copy['text'] = final_data_copy['gen_data'].apply(lambda x:x[1])
final_data_copy = final_data_copy[[i for i in final_data_copy if i != "gen_data"]]

In [9]:
final_data_copy.head(50)

,session_id,action_type,text
0,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,item_view,Satin Pajama sets shawl collar allover print n...
1,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,item_view,Carter's 9-12 Months Girls Girls 0-24 mos dress
2,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,item_view,Lonxu Pink With Pajama sets Black Trim Silk Sa...
3,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,item_add_to_cart_tap,Pajama sets Lonxu Pink With Black Trim Silk Sa...
4,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,buy_comp,Lonxu Pink With Black Trim Silk Satin Pajama s...
5,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,item_view,Boohoo Satin Pajama Set Size Pajama sets 4-6
6,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,query,dknny hipsters
6,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,item_view,BRAND Panties NEW DKNY Hipsters Size L
7,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,item_view,Boohoo Satin Pajama Set Pajama sets Size 4-6
8,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,item_view,pajama Pant suit set


In [7]:
final_data_copy[final_data_copy['action_type'] == 'label'].to_csv("Only_label_query_candidate.csv",index = None)

### Combine the candidate data that has been labeled with labels to generate instruction data

In [41]:
# 3 narrowings, actually 2, because label already counts as one
# 1 substitute
# 2 complements
# 4 irrelevants
all_query_set = set(final_data_copy[final_data_copy['action_type'] == "query"]['text'])
label_candidate = pd.read_csv("overall_labeled_candidate.csv")
def generate_candidate_str(row):
    option_list = []
    if row["narrowing_0"] == row["substitute_0"] or row["narrowing_1"] == row["substitute_0"]:
        row["substitute_0"] = random.sample(all_query_set, 1)
    if row["narrowing_0"] == row["narrowing_1"]:
        row["narrowing_1"] = row["substitute_0"]
        row["substitute_0"] = random.sample(all_query_set, 1)
    for (single_rel,times) in [('text', 1),('narrowing', 2),('substitute', 1),('complement', 2)]: # , ('irrelevant', 3)
        for i in range(times):
            if single_rel == "text":
                option_list.append([row[single_rel],1])
            elif single_rel == "narrowing":
                option_list.append([row[f"{single_rel}_{i}"],1])
            else:
                option_list.append([row[f"{single_rel}_{i}"],0])
    option_list += [[i,0] for i in random.sample(all_query_set, 4)]
    random.shuffle(option_list)
    label_idx_list = [str(idx + 1) for idx,i in enumerate(option_list) if i[1] == 1]
    option_list = [f"{idx + 1}. {i[0]}" for idx,i in enumerate(option_list)]
    option_str = "\n".join(option_list)
    return [option_str, ",".join(label_idx_list)]
    
label_candidate['all_candidate'] = label_candidate.apply(lambda row:generate_candidate_str(row), axis = 1)
label_candidate['out_put'] = label_candidate['all_candidate'].apply(lambda x:x[1])
label_candidate['all_candidate'] = label_candidate['all_candidate'].apply(lambda x:x[0])

In [42]:
# check
tmp_idx = 1
print("text：",label_candidate.iloc[tmp_idx]['text'])
print("all_candidate：",label_candidate.iloc[tmp_idx]['all_candidate'])
print("out_put：",label_candidate.iloc[tmp_idx]['out_put'])
label_candidate

text： tory burch brown leather flats
all_candidate： 1. boys 2t 5t shorts
2. tory burch leather ankle boots
3. catch surf tank top
4. burberry brown leather shoes
5. tory burch brown boots
6. 4 port tv splitter
7. 11.5 men nike shoes
8. tory burch brown leather flats
9. tory burch
10. tory burch brown shoulder bag
out_put： 2,4,8


,session_id,action_type,text,narrowing_0,narrowing_1,substitute_0,substitute_1,complement_0,complement_1,all_candidate,out_put
0,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,label,dress,evening dress,ladies dress,skirtwear,outfit,shoes,shoes,1. skirtwear\n2. nike air max 90 white\n3. ver...,"5,6,10"
1,000b026d453227fb863e3748ca1d42695e80bd9505e5cb...,label,tory burch brown leather flats,burberry brown leather shoes,tory burch leather ankle boots,tory burch,tory burch gold earrings,tory burch brown boots,tory burch brown shoulder bag,1. boys 2t 5t shorts\n2. tory burch leather an...,"2,4,8"
2,000d75ba39872a973eadbec07c37f9d20d6d4f17a6ce42...,label,nintendo nes joystick,nes controller,classic nintendo controller,nintendo switch controller,nintendo switch controller,nintendo nes games,nintendo nes games,1. little boys t shirts 2t 5t\n2. nintendo swi...,"4,5,10"
3,0013185687c43feb237d102c7ccef2260942149c77981f...,label,kobe american flag sweater,nba team flag hoodie kobe,nba,patriotic hoodie sweater,red jersey sweater,black basketball shorts,nba jersey and hat,1. nba jersey and hat\n2. patriotic hoodie swe...,"7,8,10"
4,00165b62631c6195833646f64b39dda2ce9c0db0d46c7a...,label,pink bras 34b,ladies plus size bra 34b,pink lingerie,pink women's socks,lingerie sets,purple panties 34a,light pink underwear sets,1. pink bras 34b\n2. red women's blouse\n3. pu...,"1,4,7"
...,...,...,...,...,...,...,...,...,...,...,...
28567,fff5001dc7bb5ae0452292c5afee57067cff7579ac1595...,label,merrell mens boots size 11,merrell men's boot,Merrell men's shoes size 11 wide width,Merrell mens shoes waterproof,merrell mens shoes size 11,merrell mens shoes size 11 waterproof,merrell men shirts size,1. wolverine infinity watch\n2. merrell mens b...,"2,6,8"
28568,fff889fa6e2f4471b8cff426ab9caf1a983919604b1f51...,label,happy planner,daily organizer planner,journal,[louis vuitton alma],bullet journal,stickers for organization,daily planner stickers,1. turquoise ring size 7\n2. daily planner sti...,"4,5,6"
28569,fff8e5a2c730bb24babacc3ba0f2103edf55c77f0a4d9f...,label,gucci leather wallet,prada men's leather wallet,hermes supple leather wallet,prada card holder,gucci belt,gucci belt,gucci card holder,1. hermes supple leather wallet\n2. jay tuffle...,"1,5,6"
28570,fff956c8e0529b544985661ed57ea14f9fb142ff7287cd...,label,lululemon high rise athletic pants,lululemon athletic capris,athleta leggings,lululemon yoga shorts,athleta yogalife pants,lululemon tops,lululemon top,1. lululemon athletic capris\n2. athleta leggi...,"1,2,3"


In [43]:
seq_data = copy.deepcopy(final_data_copy)
# merge_data = pd.merge(merge_data, label_candidate, how = "left")
actionType_new_dict = {"buy_comp":"Add product '{}' to cart",
                      "item_add_to_cart_tap":"Purchase product '{}'",
                      "item_view":"Click on product  '{}'",
                       "query":"Query keyword '{}'",
                      "label":""}
seq_data['add_type_title'] = seq_data.apply(lambda row:actionType_new_dict[row['action_type']].format(row['text']),axis = 1)


def generate_sequence(group):
    return group[:-1].tolist()
seq_data = seq_data.groupby('session_id')['add_type_title'].apply(generate_sequence).reset_index()
seq_data.columns = ['session_id', 'add_type_title']

seq_data = pd.merge(seq_data, label_candidate[['session_id','all_candidate','out_put']], on = "session_id", how = "left")
seq_data

,session_id,add_type_title,all_candidate,out_put
0,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,[Click on product 'Satin Pajama sets shawl co...,1. skirtwear\n2. nike air max 90 white\n3. ver...,"5,6,10"
1,000b026d453227fb863e3748ca1d42695e80bd9505e5cb...,[Click on product 'Tory Belts Burch leather b...,1. boys 2t 5t shorts\n2. tory burch leather an...,"2,4,8"
2,000d75ba39872a973eadbec07c37f9d20d6d4f17a6ce42...,[Click on product 'NEW YORK KNICKS CARMELO Mc...,1. little boys t shirts 2t 5t\n2. nintendo swi...,"4,5,10"
3,0013185687c43feb237d102c7ccef2260942149c77981f...,[Click on product 'Chaps Ralph Lauren Sweater...,1. nba jersey and hat\n2. patriotic hoodie swe...,"7,8,10"
4,00165b62631c6195833646f64b39dda2ce9c0db0d46c7a...,[Query keyword 'victorias secret 34b bralette'...,1. pink bras 34b\n2. red women's blouse\n3. pu...,"1,4,7"
...,...,...,...,...
28567,fff5001dc7bb5ae0452292c5afee57067cff7579ac1595...,[Click on product 'Oakley Briefcase Bag Brief...,1. wolverine infinity watch\n2. merrell mens b...,"2,6,8"
28568,fff889fa6e2f4471b8cff426ab9caf1a983919604b1f51...,"[Query keyword 'happy planner stickers', Click...",1. turquoise ring size 7\n2. daily planner sti...,"4,5,6"
28569,fff8e5a2c730bb24babacc3ba0f2103edf55c77f0a4d9f...,"[Query keyword 'burberry cashmere scarf', Clic...",1. hermes supple leather wallet\n2. jay tuffle...,"1,5,6"
28570,fff956c8e0529b544985661ed57ea14f9fb142ff7287cd...,[Click on product 'Lululemon On The XS Fly Pa...,1. lululemon athletic capris\n2. athleta leggi...,"1,2,3"


In [44]:
instruction_data = pd.DataFrame()
template = "You are a user on an online shopping platform. You make queries and click on products to eventually find the product you want and make your purchase. \nSuppose you have just performed the following sequence of actions of queries, clicks, and purchases. What is most likely to be the keyword of your next query? \nYou are given a numbered list of ten candidate queries. Select three from the list that you think most likely. Output ONLY three numbers separated with comma. Do not give explanations. \n\nAction Sequence:\n{}\n\nCandidate Query List:\n{}\nOutput (answer in three comma-separated numbers): "
instruction_data['prompt'] = seq_data.apply(lambda row:template.format("\n".join(row['add_type_title']), row['all_candidate']), axis = 1)
instruction_data['output'] = seq_data['out_put']

In [45]:
instruction_data.to_csv("session_rec_task.csv",index = None)